# pandas

> `pandas` is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool,
built on top of the Python programming language.

`pandas` is one of the most powerful tools in the entire data science ecosystem. It allows us to work with tabular and multidimensional data with extreme ease and flexibility, and is incredibly easy to jump into. 


## Series & DataFrames

To start we will learn about the two primary data structures that `pandas` defines: series and DataFrames.

### Series

`pandas` series (in code `pd.Series`) are arrays of data (much like a NumPy array). They usually contain elements that are of the same type, but this is not a requirement. We can generally think of a series as a column of data, where we can label rows. Let's make a simple series from random NumPy data by simply passing a NumPy array into the constructor:

In [1]:
import numpy as np
import pandas as pd  # it is standard to import pandas with the alias pd


np.random.seed(1)
random_data = np.random.randint(0,10,10)
random_series = pd.Series(random_data)  # create a series containing 10 random integers in the half-open set [0,10)
random_series

0    5
1    8
2    9
3    5
4    0
5    0
6    1
7    7
8    6
9    9
dtype: int32

Displaying a series will show both the row-numbers (also called the index) on the left, with the data to the right. As expected the row numbers range from 0 to 9, and the numbers in our column seem sufficiently random. `pandas` also determined that the `dtype` for the series should be 64-bit integer (since that was the type that NumPy generated using `randint`).

We can access the elements of our series in the same familiar ways as before, but there are also some new ways we need to understand.

In [2]:
(
    random_series[1],      # standard access, but deprecated! don't access a series like this!
    random_series.loc[1],  # access the row with index **label** 1 (i.e., the row with index equal to one)
    random_series.iloc[1]  # access the row with index **position** 1 (i.e., the 2nd row)
)

(8, 8, 8)

Why would we need these new methods for accessing data from a series? The difference between these access methods will make more sense if we add different *labels* to our index. We can access the index of a series easily:

In [3]:
random_series.index

RangeIndex(start=0, stop=10, step=1)

Because we initialized the series with a list, `pandas` efficiently just represents the index using an integer range to represent row numbers. Numerical labeling is the default behavior for `pandas`. We can manually override this index with a new one of the same size. Here we are going to manually replace the existing index with a new one from a list of labels.

In [4]:
random_series.index = ['a','b','c','d','e','f','g','h','i','j']  # this must have the same length as the previous index!
random_series

a    5
b    8
c    9
d    5
e    0
f    0
g    1
h    7
i    6
j    9
dtype: int32

Now we can access the series' elements by their index labels:

In [5]:
print(random_series.loc['b']) # access and print the row with index label 'b'
print(random_series.iloc[1]) # access and print the second row (same as the row with index label 'b')
print(random_series.iloc[2]) # access and print the third row

8
8
9


We can also access an element directly by the index label *if and only if the label is a string!*

In [6]:
random_series.b # equivalent to random_series.loc['b']

8

We can also slice series just like lists:

In [7]:
random_series.iloc[3:7]

d    5
e    0
f    0
g    1
dtype: int32

But note that because our index is no longer a range, we cannot slice `loc` using integers. Instead, we can slice `loc` using the labels!

In [9]:
# This would return an error; we need to slide loc by the actual labels (in this case, strings)
# print(random_series.loc[3:7])
print(random_series.loc['d':'g'])

d    5
e    0
f    0
g    1
dtype: int32


We can perform operations on series like we can with arrays and lists, but we should take care to *never write an explicit loop*. Instead we should use the `apply` method to apply functions to a series. This is more pythonic and in some cases faster.

In [10]:
random_series.apply(lambda x: x * x)  # returns a new series with each row squared

a    25
b    64
c    81
d    25
e     0
f     0
g     1
h    49
i    36
j    81
dtype: int64

Assuming that two series share the same index, we can perform mathematical operations like addition using two series. 

In [11]:
other_random_series = pd.Series(np.random.randint(0,10,10), index=['a','b','c','d','e','f','g','h','i','j'])
random_series + other_random_series

a     7
b    12
c    14
d     7
e     4
f     2
g     5
h    14
i    13
j    18
dtype: int32

What happens when our indices do not match?

In [12]:
other_random_series = pd.Series(np.random.randint(0,10,10))
random_series + other_random_series

a   NaN
b   NaN
c   NaN
d   NaN
e   NaN
f   NaN
g   NaN
h   NaN
i   NaN
j   NaN
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
6   NaN
7   NaN
8   NaN
9   NaN
dtype: float64

When performing an operation like addition with two series, `pandas` will attempt to add elements that have common labels. Any labels that don't have a pair are effectively converted to `NaN`, or a value indicating "not a number".

If we ever need to change our series into a list or NumPy array, we can use the methods `to_list` and `to_numpy`:

In [13]:
random_series.to_list(), random_series.to_numpy()

([5, 8, 9, 5, 0, 0, 1, 7, 6, 9], array([5, 8, 9, 5, 0, 0, 1, 7, 6, 9]))

## DataFrames




Having individual columns of data is much less useful than actually having a table of data, especially if all of the series are aligned on their index! A dataframe is a tabular piece of data with a row index just like series, but also contains a column index. A common way of creating DataFrames is from dictionaries, mapping column names to lists/arrays/series of data.

In [14]:
platonic_properties = {
    'vertices': [4, 8, 6, 20, 12],
    'edges': [6, 12, 12, 30, 30],
    'faces': [4, 6, 8, 12, 20],
}
names = ['tetrahedron', 'cube', 'octahedron', 'dodecahedron', 'icosahedron']

platonic_solids = pd.DataFrame(platonic_properties, index=names)
platonic_solids

,vertices,edges,faces
tetrahedron,4,6,4
cube,8,12,6
octahedron,6,12,8
dodecahedron,20,30,12
icosahedron,12,30,20


This dataframe contains properties of platonic solids. It has an index (again, these are the row labels) which we set to the names of the platonic solid. The dataframe also has 3 columns: vertices, edges, and faces.

We can call the `info` function of the dataframe to observe some of its properties:

In [15]:
platonic_solids.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, tetrahedron to icosahedron
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   vertices  5 non-null      int64
 1   edges     5 non-null      int64
 2   faces     5 non-null      int64
dtypes: int64(3)
memory usage: 160.0+ bytes


We can also use the `describe` method to numerically summarize the table, or the `memory_usage` method to look at the specific memory usage per column.

We can access data by row, by column, or by individual elements directly. We can access all of these pieces of data in multiple ways. Generally, when accessing *rows* of data, we will use the `loc` or `iloc` attributes. When accessing columns of data we will use the column name directly.

In [16]:
# access the data for a dodecahedron
platonic_solids.loc['dodecahedron']

vertices    20
edges       30
faces       12
Name: dodecahedron, dtype: int64

In [17]:
# access the number of edges for each solid
platonic_solids.loc[:, 'edges']

tetrahedron      6
cube            12
octahedron      12
dodecahedron    30
icosahedron     30
Name: edges, dtype: int64

In [18]:
# we can abbreviate selecting an entire column
platonic_solids['edges']  # or even platonic_solids.edges

tetrahedron      6
cube            12
octahedron      12
dodecahedron    30
icosahedron     30
Name: edges, dtype: int64

In [19]:
# access the number of vertices that a dodecahedron has
platonic_solids.loc['dodecahedron', 'vertices']

20

The true power of `pandas` comes with selecting data. Selecting slices and individual elements are fine, but we usually need more than that. Let's say we want to select all platonic solids with 12 edges. To do so we need to *create a mask*. A mask is list/array/series of booleans that indicates whether or not to select the value at its associated position. Creating a mask is as simple as just applying some comparison directly to a series/column:

In [20]:
edge_mask = platonic_solids.edges == 12
edge_mask

tetrahedron     False
cube             True
octahedron       True
dodecahedron    False
icosahedron     False
Name: edges, dtype: bool

We can use this mask to then access the rows that satisfy the comparison above

In [21]:
platonic_solids.loc[edge_mask]

,vertices,edges,faces
cube,8,12,6
octahedron,6,12,8


## Exercise

Create a mask to select rows where the number of vertices is greater than the number of faces.

In [27]:
## Exercise

vertices_mask = platonic_solids['vertices'] > platonic_solids['faces']
vertices_mask

tetrahedron     False
cube             True
octahedron      False
dodecahedron     True
icosahedron     False
dtype: bool

`pandas` makes it super easy to load data from a file. We have a file in this repository called `nj_counties` that contains some census data detailing the population of each county of NJ for the year 2020, and estimates and projections for 2021 and 2022. We can tell `pandas` to automatically read a CSV into a dataframe using the `read_csv` method.

In [28]:
nj_county_data = pd.read_csv('data/nj_counties.csv')
nj_county_data

,County,2020,2021,2022
0,Atlantic County,274172,275130,275638
1,Bergen County,953617,954879,952997
2,Burlington County,461648,464411,466103
3,Camden County,523074,524124,524907
4,Cape May County,95040,95768,95634
5,Cumberland County,153692,152089,151356
6,Essex County,859924,854121,849477
7,Gloucester County,302554,304620,306601
8,Hudson County,721832,703447,703366
9,Hunterdon County,128786,129668,129777


Let's organize this dataframe a little, to make it a little easier to work with. We want to move the column "County" to be the index of the dataframe, and we want to make sure that the column names are actual integers. When we parse columns from a CSV, `pandas` will automatically make them strings. This is usually OK, but since we are working with years, which are conventionally integers, we want to make that conversion.

For the index we can tell `pandas` to use a column as the index using the method `set_index`. To update the columns, we can tell `pandas` to reinterpret the labels as integers and overwrite them with the integer version.

In [29]:
nj_county_data.set_index('County', inplace=True)  # pandas operations by default returns copies, but many operations can be told to be inplace!
nj_county_data.columns = nj_county_data.columns.astype(int)
nj_county_data

,2020,2021,2022
County,,,
Atlantic County,274172,275130,275638
Bergen County,953617,954879,952997
Burlington County,461648,464411,466103
Camden County,523074,524124,524907
Cape May County,95040,95768,95634
Cumberland County,153692,152089,151356
Essex County,859924,854121,849477
Gloucester County,302554,304620,306601
Hudson County,721832,703447,703366


By setting the index to the county, we can now access rows by their county, and now we can access yearly data more intuitively:

In [30]:
nj_county_data.loc['Camden County', 2022]

524907

Let's add two columns to our dataframe called "Change" and "Percent Change" (respectively the difference from 2020 to 2022, and that difference divided by the counts of 2020). Because columns are just series (that share the same index!) we can easily use mathematical operations to create new series! For example:

In [31]:
nj_county_data[2022] - nj_county_data[2020]

County
Atlantic County       1466
Bergen County         -620
Burlington County     4455
Camden County         1833
Cape May County        594
Cumberland County    -2336
Essex County        -10447
Gloucester County     4047
Hudson County       -18466
Hunterdon County       991
Mercer County        -5753
Middlesex County       104
Monmouth County       1327
Morris County         2767
Ocean County         17313
Passaic County       -9470
Salem County           275
Somerset County       2150
Sussex County         2169
Union County         -3802
Warren County         1413
dtype: int64

We can assign this expression to a new column; all we need to do is use the subscript operator to access the new column and assign it some data that either has the same index as the dataframe, or the same length:

In [32]:
nj_county_data['Change'] = nj_county_data[2022] - nj_county_data[2020]
nj_county_data['Percent Change'] = 100.0 * nj_county_data['Change'] / nj_county_data[2020]
nj_county_data

,2020,2021,2022,Change,Percent Change
County,,,,,
Atlantic County,274172,275130,275638,1466,0.534701
Bergen County,953617,954879,952997,-620,-0.065016
Burlington County,461648,464411,466103,4455,0.965021
Camden County,523074,524124,524907,1833,0.350428
Cape May County,95040,95768,95634,594,0.625000
Cumberland County,153692,152089,151356,-2336,-1.519923
Essex County,859924,854121,849477,-10447,-1.214875
Gloucester County,302554,304620,306601,4047,1.337612
Hudson County,721832,703447,703366,-18466,-2.558213


We can more clearly see how each county's population has changed - let's sort the dataframe according to the percent changes, in descending order:

In [33]:
nj_county_data.sort_values(by='Percent Change', ascending=False, inplace=True)
nj_county_data

,2020,2021,2022,Change,Percent Change
County,,,,,
Ocean County,638422,649741,655735,17313,2.711843
Sussex County,143915,145645,146084,2169,1.507140
Gloucester County,302554,304620,306601,4047,1.337612
Warren County,109513,110494,110926,1413,1.290258
Burlington County,461648,464411,466103,4455,0.965021
Hunterdon County,128786,129668,129777,991,0.769494
Cape May County,95040,95768,95634,594,0.625000
Somerset County,344725,346331,346875,2150,0.623686
Morris County,508384,510444,511151,2767,0.544274


While looking at raw changes are useful, it can be just as useful looking at percentage changes. Percentages can normalize a dataset, and can help highlight substantial fluctuations within the data (e.g. a change of 20k people in Bergen County is significantly less impactful than a change of 20k people in Salem County). 

## Exercise

We want to take our NJ population data and filter it on multiple criteria. We can apply logical combinations of masks using the `&` (element-wise AND) and `|` (element-wise OR) operators. These are considered "element-wise" operations in the sense that we perform operations on each element of a mask.

We want to get all counties having `Percent Change` with an absolute value grater than 1 and a 2022 estimated population greater than 500000.0. Create a mask for each of those two criteria, and then create a third mask that is created by joining the first two using a *bitwise-and* operation.

In [44]:
abs_greater_than_1_mask = np.abs(nj_county_data['Percent Change']) > 1
population_2022_greater_than_500000 = nj_county_data[2022] > 500000

intersection_mask = population_2022_greater_than_500000 & abs_greater_than_1_mask

intersection_mask

nj_county_data[intersection_mask]

,2020,2021,2022,Change,Percent Change
County,,,,,
Ocean County,638422,649741,655735,17313,2.711843
Essex County,859924,854121,849477,-10447,-1.214875
Passaic County,523406,518345,513936,-9470,-1.809303
Hudson County,721832,703447,703366,-18466,-2.558213
